In [1]:
import pandas as pd
import numpy as np
import os
import sys

In [2]:
# Load the data with all questions with citations
cite_data = pd.read_csv("query_results_metrics_v2/query_results_v2_full.csv")

print("Columns in cite_data: ", cite_data.columns)

Columns in cite_data:  Index(['use_image', 'retrieved_docs_count', 'use_few_shot', 'query_id',
       'user_query', 'generated_question', 'generated_answer', 'citations',
       'citation_1_text', 'citation_2_text', 'citation_3_text',
       'citation_4_text'],
      dtype='object')


In [3]:
display(cite_data)

,use_image,retrieved_docs_count,use_few_shot,query_id,user_query,generated_question,generated_answer,citations,citation_1_text,citation_2_text,citation_3_text,citation_4_text
0,False,1,False,01JF744592EPH2XXTYXCZMVQP4,I need some exercises involving matrix multipl...,A science museum offers tickets at different p...,43850,674dcdb4b00b977d048c931b,The admission tickets to the Singapore Zoo are...,NaN,NaN,NaN
1,False,1,False,01JF744592VMGDKR4BKX9T8TK7,Where can I find practice questions on multipl...,The entrance tickets to the National Aquarium ...,28010,674dcdb4b00b977d048c931b,The admission tickets to the Singapore Zoo are...,NaN,NaN,NaN
2,False,1,False,01JF744592VJC0GYVQVWTH0RAP,Can you provide practice problems on multiplyi...,The tickets for a city museum are priced at $4...,"Totalrevenue=$18,950",674dcdb4b00b977d048c931b,The admission tickets to the Singapore Zoo are...,NaN,NaN,NaN
3,False,1,False,01JF744592K2JTR2NG3HSHCVAS,Do you have any sample questions on multiplyin...,The entry tickets for a local amusement park a...,40340,674dcdb4b00b977d048c931b,The admission tickets to the Singapore Zoo are...,NaN,NaN,NaN
4,False,1,False,01JF744SEDV98F8FMMT5642XVN,"I need exercises involving calculating mean, m...",Sophia collects data on the number of hours sh...,3hours,674dcd9cb00b977d048c92ff,"Ethan measures the amount of rain, in millimet...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
568,False,4,True,01JF77SWDGVGRQ9CXG1NDG03ZB,I need exercises involving evaluating algebrai...,An equation is given as \( z = 3py - 4m \). De...,\(p=\frac{-2}{9}\),674dce18b00b977d048c939f,"Given \( a = 2bv + xk \), find \( x \) when \(...",NaN,NaN,NaN
569,False,4,True,01JF77T7P4Q10B5ASTZRCR20PE,Could you give me some problems to practice ev...,Consider the formula \( c = 3dp - yz \). Deter...,\frac{13}{4},674dce18b00b977d048c939f,"Given \( a = 2bv + xk \), find \( x \) when \(...",NaN,NaN,NaN
570,False,4,True,01JF77SXWHDHP98KVQ06RYRYA8,Where can I find practice questions on evaluat...,"Given the formula \( a = b(m - xc) \), find \(...",x=\frac{10}{3},674dce18b00b977d048c939f,"Given \( a = 2bv + xk \), find \( x \) when \(...",NaN,NaN,NaN
571,False,4,True,01JF77T6G9C3DN19THBD6HG2NS,Do you have any sample questions on evaluating...,"If the equation \( z = mwn + p \) holds true, ...",n=-2,674dce18b00b977d048c939f,"Given \( a = 2bv + xk \), find \( x \) when \(...",NaN,NaN,NaN


In [4]:
# Columns to group by
group_columns = ['use_image', 'retrieved_docs_count', 'use_few_shot']

# Split dataframe into groups
split_dataframes = {
    group_key: group_df.reset_index(drop=True)
    for group_key, group_df in cite_data.groupby(group_columns)
}

files = []
# Display the resulting dataframes
for group_key, group_df in split_dataframes.items():
    print(f"Group: {group_key}")
    
    # Save the group to a csv file
    group_df.to_csv(f"query_results_metrics_v2/query_results_v2_{group_key}.csv", index=False)
    files.append(f"query_results_metrics_v2/query_results_v2_{group_key}.csv")
    


Group: (False, 1, False)
Group: (False, 1, True)
Group: (False, 2, False)
Group: (False, 2, True)
Group: (False, 3, False)
Group: (False, 3, True)
Group: (False, 4, False)
Group: (False, 4, True)
Group: (True, 1, False)
Group: (True, 1, True)
Group: (True, 2, False)
Group: (True, 2, True)
Group: (True, 3, False)
Group: (True, 3, True)
Group: (True, 4, False)
Group: (True, 4, True)


# Calculate New Citation Groundness Score

In [5]:
import pandas as pd
from trust_eval import _run_nli_gpt4

def calculate_citation_f1(data: pd.DataFrame):
    length_A_r = 0
    CR_score = 0
    CP_score = 0

    for _, row in data.iterrows():   # each user query
        claim = row["generated_question"]
        docs = [row["citation_1_text"], row["citation_2_text"], row["citation_3_text"], row["citation_4_text"]]
        
        if claim == "Error generating output": # Model did not generate a response
            continue

        length_A_r += 1

        # Calculate CR score for this user query
        docs_text = [doc for doc in docs if isinstance(doc, str) and doc.strip()]
        concatenated_docs = '| Question: '.join(docs_text)
        CR_score += _run_nli_gpt4(concatenated_docs, claim) # Either 1 or 0

        # Calculate CP score for this user query
        local_CP_score = 0
        local_CP_length = 0
        for doc in docs:
            if isinstance(doc, str) and doc.strip():
                local_CP_length += 1
                local_CP_score += _run_nli_gpt4(doc, claim)

        if local_CP_length > 0:
            CP_score += (local_CP_score / local_CP_length)

    overall_CR_score = CR_score / length_A_r if length_A_r > 0 else 0
    overall_CP_score = CP_score / length_A_r if length_A_r > 0 else 0
    overall_CF1_score = 2 * (overall_CR_score * overall_CP_score) / (overall_CR_score + overall_CP_score) if (overall_CR_score + overall_CP_score) > 0 else 0

    return {
        "citation_rec": 100 * overall_CR_score,
        "citation_prec": 100 * overall_CP_score,
        "citation_f1": 100 * overall_CF1_score,
    }

/opt/miniconda3/envs/cite/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/opt/miniconda3/envs/cite/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
citation_results = {}

for file in files:
    config = file.split("_")[-1].split(".")[0]
    
    data = pd.read_csv(file)
    citation_results[config] = calculate_citation_f1(data)

In [7]:
print(citation_results)
# 'use_image', 'retrieved_docs_count', 'use_few_shot':  

# Save the results to a JSON file
import json

with open("query_results_metrics_v2/citation_results_v2.json", "w") as f:
    json.dump(citation_results, f, indent=4)
    

{'(False, 1, False)': {'citation_rec': 86.11111111111111, 'citation_prec': 94.44444444444444, 'citation_f1': 90.08547008547009}, '(False, 1, True)': {'citation_rec': 66.66666666666666, 'citation_prec': 61.111111111111114, 'citation_f1': 63.76811594202899}, '(False, 2, False)': {'citation_rec': 71.42857142857143, 'citation_prec': 65.71428571428571, 'citation_f1': 68.45238095238095}, '(False, 2, True)': {'citation_rec': 69.44444444444444, 'citation_prec': 59.72222222222222, 'citation_f1': 64.21744324970132}, '(False, 3, False)': {'citation_rec': 65.71428571428571, 'citation_prec': 59.523809523809526, 'citation_f1': 62.46605105920695}, '(False, 3, True)': {'citation_rec': 61.111111111111114, 'citation_prec': 48.148148148148145, 'citation_f1': 53.86064030131826}, '(False, 4, False)': {'citation_rec': 69.44444444444444, 'citation_prec': 53.24074074074075, 'citation_f1': 60.27253668763103}, '(False, 4, True)': {'citation_rec': 69.44444444444444, 'citation_prec': 64.12037037037037, 'citation_

# Measure diversity in generated questions

In [8]:
import jellyfish

# Measure string metrics between the generated_question and each citation_text
def calculate_string_similarities(data: pd.DataFrame):
    
    # Initialize lists to store the similarity scores for each user query
    damerau_levenshtein_sims = []
    jaro_winkler_sims = []

    for _, row in data.iterrows():
        claim = row["generated_question"]
        docs = [row["citation_1_text"], row["citation_2_text"], row["citation_3_text"], row["citation_4_text"]]

        damerau_levenshtein_scores = []
        jaro_winkler_scores = []

        for doc in docs:
            if isinstance(doc, str) and doc.strip():
                # Calculate normalized Damerau-Levenshtein difference
                damerau_levenshtein = jellyfish.damerau_levenshtein_distance(claim, doc)
                max_len = max(len(claim), len(doc))
                damerau_levenshtein_diff = (damerau_levenshtein / max_len) if max_len > 0 else 0
                damerau_levenshtein_scores.append(damerau_levenshtein_diff)

                # Calculate Jaro-Winkler difference: 1 - similarity
                jaro_winkler_diff = 1 - jellyfish.jaro_winkler_similarity(claim, doc)
                jaro_winkler_scores.append(jaro_winkler_diff)

        # Average the similarities if there are valid citations
        if damerau_levenshtein_scores:
            damerau_levenshtein_sims.append(sum(damerau_levenshtein_scores) / len(damerau_levenshtein_scores))
        else:
            damerau_levenshtein_sims.append(0)

        if jaro_winkler_scores:
            jaro_winkler_sims.append(sum(jaro_winkler_scores) / len(jaro_winkler_scores))
        else:
            jaro_winkler_sims.append(0)

    # Add the similarity columns to the dataframe
    data["damerau_levenshtein_sim"] = damerau_levenshtein_sims
    data["jaro_winkler_sim"] = jaro_winkler_sims
    
    # Aggregate each similarity metric across all user queries
    damerau_levenshtein_avg = data["damerau_levenshtein_sim"].mean()
    jaro_winkler_avg = data["jaro_winkler_sim"].mean()

    return data, damerau_levenshtein_avg, jaro_winkler_avg


In [9]:
string_diversity_results = {}

for file in files:
    config = file.split("_")[-1].split(".")[0]
    
    data = pd.read_csv(file)
    data, d_l_avg, j_w_avg = calculate_string_similarities(data)
    string_diversity_results[config] = {
        "damerau_levenshtein_avg": d_l_avg,
        "jaro_winkler_avg": j_w_avg
    }
    data.to_csv(f"{file.split('.')[0]}_string_similarities.csv", index=False)
    
print(string_diversity_results)

# Save the results to a JSON file
with open("query_results_metrics_v2/string_diversity_results.json", "w") as f:
    json.dump(string_diversity_results, f, indent=4)

{'(False, 1, False)': {'damerau_levenshtein_avg': 0.5301238036989334, 'jaro_winkler_avg': 0.2726459698495806}, '(False, 1, True)': {'damerau_levenshtein_avg': 0.5987697131610795, 'jaro_winkler_avg': 0.2874572162097641}, '(False, 2, False)': {'damerau_levenshtein_avg': 0.613509420756076, 'jaro_winkler_avg': 0.3122203885346307}, '(False, 2, True)': {'damerau_levenshtein_avg': 0.6861243363465646, 'jaro_winkler_avg': 0.34660558193130986}, '(False, 3, False)': {'damerau_levenshtein_avg': 0.6183104551410306, 'jaro_winkler_avg': 0.3212183913662756}, '(False, 3, True)': {'damerau_levenshtein_avg': 0.6637922267556979, 'jaro_winkler_avg': 0.3285041802380286}, '(False, 4, False)': {'damerau_levenshtein_avg': 0.6220952078262695, 'jaro_winkler_avg': 0.30813226921266773}, '(False, 4, True)': {'damerau_levenshtein_avg': 0.6125928117812158, 'jaro_winkler_avg': 0.30783776309282107}, '(True, 1, False)': {'damerau_levenshtein_avg': 0.5996655627993822, 'jaro_winkler_avg': 0.284516070667102}, '(True, 1, Tr

# Generate JSON for trust-align's Grounded Refusal Eval

In [14]:
base_JSON_file = "emath_qns_eval_data.json"

# Load the base JSON file
with open(base_JSON_file, "r") as f:
    base_JSON = json.load(f)
    
# Loop through file in files and update the base JSON file's output field
for file in files:
    config = file.split("_")[-1].split(".")[0]
    
    # Read the CSV file into a DataFrame
    df = pd.read_csv(file)
    
    # Create a dictionary for quick lookups:
    # { user_query: generated_question }
    update_map = dict(zip(df["user_query"], df["generated_question"]))
    
    # We will modify a copy of base_JSON to preserve original if needed
    updated_JSON = {"data": []}
    
    # Iterate over each entry in base_JSON["data"]
    for entry in base_JSON["data"]:
        # If the question in the JSON matches a user_query in CSV
        q = entry.get("question", "")
        if q in update_map:
            # Update the output field with the generated_question
            entry["output"] = update_map[q]
        
        # Append the (potentially) updated entry to updated_JSON
        updated_JSON["data"].append(entry)
    
    # Save the updated JSON to a new file with the config name
    output_file = f"query_results_metrics_v2/updated_data_{config}.json"
    with open(output_file, "w") as out_f:
        json.dump(updated_JSON, out_f, indent=4)

    print(f"Updated JSON saved to {output_file}")

Updated JSON saved to query_results_metrics_v2/updated_data_(False, 1, False).json
Updated JSON saved to query_results_metrics_v2/updated_data_(False, 1, True).json
Updated JSON saved to query_results_metrics_v2/updated_data_(False, 2, False).json
Updated JSON saved to query_results_metrics_v2/updated_data_(False, 2, True).json
Updated JSON saved to query_results_metrics_v2/updated_data_(False, 3, False).json
Updated JSON saved to query_results_metrics_v2/updated_data_(False, 3, True).json
Updated JSON saved to query_results_metrics_v2/updated_data_(False, 4, False).json
Updated JSON saved to query_results_metrics_v2/updated_data_(False, 4, True).json
Updated JSON saved to query_results_metrics_v2/updated_data_(True, 1, False).json
Updated JSON saved to query_results_metrics_v2/updated_data_(True, 1, True).json
Updated JSON saved to query_results_metrics_v2/updated_data_(True, 2, False).json
Updated JSON saved to query_results_metrics_v2/updated_data_(True, 2, True).json
Updated JSON s

# Prepare JSON for entailment evaluation dataset

In [10]:
def generate_json_data(data: pd.DataFrame):
    json_data = []
    questions = set()

    for _, row in data.iterrows():
        citations = []
        for col in ["citation_1_text", "citation_2_text", "citation_3_text", "citation_4_text"]:
            cited_text = row[col]
            if isinstance(cited_text, str) and cited_text.strip():
                citations.append({"cited_text": cited_text, "score": None})

        if row["generated_question"] not in questions:
            json_data.append({
                "generated_question": row["generated_question"],
                "citations": citations
            })

    with open("entailment_eval.json", "w") as f:
        json.dump(json_data, f, indent=4)

In [11]:
generate_json_data(cite_data)

In [12]:
def eval_gpt4o_scores(json_data):
    match_count = 0
    total_count = 0
    
    for item in json_data:
        claim = item["generated_question"]
        for citation in item["citations"]:
            total_count += 1
            passage = citation["cited_text"]
            gpt4o_score = _run_nli_gpt4(passage, claim)
            citation["gpt4o_score"] = int(gpt4o_score)
            
            if int(gpt4o_score) == citation['score']:
                match_count += 1
                
    accuracy = match_count / total_count if total_count > 0 else 0
    
    # Save the updated JSON data
    with open("entailment_eval_gpt4o.json", "w") as f:
        json.dump(json_data, f, indent=4)
        
    return accuracy

In [13]:
# Load the JSON data
with open("entailment_eval.json", "r") as f:
    json_data = json.load(f)
    
accuracy = eval_gpt4o_scores(json_data)


Running NLI for GPT-4o...
Passage:  The admission tickets to the Singapore Zoo are $50 for an adult, $36 for a child, and $20 for a senior citizen. On a particular Tuesday, there were 212 adults, 251 children, and 15 senior citizens who visited the Singapore Zoo and on a particular Wednesday, there were 231 adults, 266 children, and 12 senior citizens who visited the Singapore Zoo. The number of visitors on the particular Tuesday and Wednesday can be represented by the matrix V = \begin{pmatrix} 212 & 251 & 15 \\ 231 & 266 & 12 \end{pmatrix}. \text{ (i) Write a } 3 \times 1 \text{ matrix, P, to represent the price of the admission tickets. (ii) Find the matrix } T = VP. \text{ (iii) Explain what each of the elements represents. (iv) Find the total amount collected from the sales of the tickets for the 2 days.}
Claim:  A science museum offers tickets at different prices: $45 for adults, $35 for students, and $25 for children. On a Saturday, the museum recorded 200 adults, 320 students, 

 2024-12-17 21:24:45,697 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Entailment Result: 1
Running NLI for GPT-4o...
Passage:  The admission tickets to the Singapore Zoo are $50 for an adult, $36 for a child, and $20 for a senior citizen. On a particular Tuesday, there were 212 adults, 251 children, and 15 senior citizens who visited the Singapore Zoo and on a particular Wednesday, there were 231 adults, 266 children, and 12 senior citizens who visited the Singapore Zoo. The number of visitors on the particular Tuesday and Wednesday can be represented by the matrix V = \begin{pmatrix} 212 & 251 & 15 \\ 231 & 266 & 12 \end{pmatrix}. \text{ (i) Write a } 3 \times 1 \text{ matrix, P, to represent the price of the admission tickets. (ii) Find the matrix } T = VP. \text{ (iii) Explain what each of the elements represents. (iv) Find the total amount collected from the sales of the tickets for the 2 days.}
Claim:  The entrance tickets to the National Aquarium are priced at $40 for adults, $28 for children, and $18 for senior citizens. On a Monday, 180 adults, 2

 2024-12-17 21:24:46,312 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Entailment Result: 1
Running NLI for GPT-4o...
Passage:  The admission tickets to the Singapore Zoo are $50 for an adult, $36 for a child, and $20 for a senior citizen. On a particular Tuesday, there were 212 adults, 251 children, and 15 senior citizens who visited the Singapore Zoo and on a particular Wednesday, there were 231 adults, 266 children, and 12 senior citizens who visited the Singapore Zoo. The number of visitors on the particular Tuesday and Wednesday can be represented by the matrix V = \begin{pmatrix} 212 & 251 & 15 \\ 231 & 266 & 12 \end{pmatrix}. \text{ (i) Write a } 3 \times 1 \text{ matrix, P, to represent the price of the admission tickets. (ii) Find the matrix } T = VP. \text{ (iii) Explain what each of the elements represents. (iv) Find the total amount collected from the sales of the tickets for the 2 days.}
Claim:  The tickets for a city museum are priced at $45 for adults, $30 for students, and $25 for children. On a particular Monday, there were 120 adults, 20

...

 2024-12-17 21:25:26,648 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Entailment Result: 1
Running NLI for GPT-4o...
Passage:  (c) Ethan picks one of these days at random. Find the probability that, on that day, the amount of rain was 3 mm or more.
Claim:  A jar contains a total of 30 marbles, of which 12 are blue and the rest are not blue. If one marble is selected at random, find the probability that it is blue.


 2024-12-17 21:25:27,339 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Entailment Result: 0
Running NLI for GPT-4o...
Passage:  The diagram is the speed-time graph for the first k seconds of the motion of an object. (a) Find the acceleration when \( t = 5 \).
Claim:  A car travels along a straight road, and its speed in meters per second over time is represented by the following speed-time graph piece: \[ \begin{array}{|c|c|} \hline \text{Time (s)} & \text{Speed (m/s)} \\ \hline 0 & 0 \\ 10 & 20 \\ 30 & 20 \\ 50 & 0 \\ \hline \end{array} \] Determine the total distance traveled by the car during these 50 seconds.


KeyboardInterrupt: 

In [39]:
print("Accuracy of GPT-4o as Entailment Judge: ", accuracy)

Accuracy of GPT-4o as Entailment Judge:  0.6190476190476191
